1: Khởi tạo thư viện và biến toàn cục

In [14]:
!pip install opencv-python mediapipe pandas

In [ ]:
import cv2
import os
import time
import mediapipe as mp
import pandas as pd
from datetime import datetime

# Đường dẫn lưu dữ liệu
BASE_PATH = r"D:\file\project\Sign-Language-Classification-v1.0\data\anphabet"

# Bảng chữ cái tiếng Việt (23 ký tự không dấu)
VIETNAMESE_ALPHABET = [
    'a', 'b', 'c', 'd', 'đ', 'e', 'g', 'h', 'i', 'k', 'l', 'm', 'n',
    'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y'
]

# Số lượng mẫu cần thu thập
NUM_SAMPLES_PER_FOLDER = 50

# Khởi tạo Mediapipe
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils


2: Định nghĩa các hàm hỗ trợ

In [16]:
def create_folders(base_path, letter):
    folder_path = os.path.join(base_path, letter)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    return folder_path

def save_landmarks_to_csv(hand_landmarks, face_landmarks, file_path):
    data = {}

    # Hand
    if hand_landmarks:
        for i, landmark in enumerate(hand_landmarks.landmark):
            data[f'hand_x_{i}'] = landmark.x
            data[f'hand_y_{i}'] = landmark.y
            data[f'hand_z_{i}'] = landmark.z
    else:
        for i in range(21):
            data[f'hand_x_{i}'] = 0.0
            data[f'hand_y_{i}'] = 0.0
            data[f'hand_z_{i}'] = 0.0

    # Face
    face_keypoints = [10, 152, 234, 454, 33, 263, 1, 61, 291, 199]
    if face_landmarks:
        for idx in face_keypoints:
            landmark = face_landmarks.landmark[idx]
            data[f'face_x_{idx}'] = landmark.x
            data[f'face_y_{idx}'] = landmark.y
            data[f'face_z_{idx}'] = landmark.z
    else:
        for idx in face_keypoints:
            data[f'face_x_{idx}'] = 0.0
            data[f'face_y_{idx}'] = 0.0
            data[f'face_z_{idx}'] = 0.0

    df = pd.DataFrame([data])
    df.to_csv(file_path, index=False)


3: Chạy ghi hình cho một ký tự cụ thể

In [17]:
def capture_for_letter(letter):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở camera!")
        return

    folder_path = create_folders(BASE_PATH, letter)
    print(f"\nBắt đầu chụp landmarks cho chữ cái: '{letter}'")

    count = 1
    file_number = 1
    start_time = time.time()

    while count <= NUM_SAMPLES_PER_FOLDER:
        ret, frame = cap.read()
        if not ret:
            print("Không đọc được khung hình!")
            break

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        hand_results = hands.process(frame_rgb)
        face_results = face_mesh.process(frame_rgb)

        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

        cv2.imshow("Capture", frame)

        if time.time() - start_time >= 2 and (hand_results.multi_hand_landmarks or face_results.multi_face_landmarks):
            file_path = os.path.join(folder_path, f"{file_number}.csv")
            save_landmarks_to_csv(
                hand_results.multi_hand_landmarks[0] if hand_results.multi_hand_landmarks else None,
                face_results.multi_face_landmarks[0] if face_results.multi_face_landmarks else None,
                file_path
            )
            print(f"[{count}/{NUM_SAMPLES_PER_FOLDER}] Đã lưu: {file_path}")
            count += 1
            file_number += 1
            start_time = time.time()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Đã thoát.")
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"✔️ Hoàn tất thu thập '{letter}' ({count-1} mẫu).")


4: Chạy thử nghiệm cho 1 ký tự

In [18]:
# Chạy thử nghiệm
# Nhấn 'q' để dừng sớm
capture_for_letter('a')  # thay 'a' bằng chữ bạn muốn thu thập



Bắt đầu chụp landmarks cho chữ cái: 'a'


d:\anaconda3\envs\mnist_project\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


[1/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\1.csv
[2/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\2.csv
[3/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\3.csv
[4/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\4.csv
[5/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\5.csv
[6/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\6.csv
[7/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\7.csv
[8/50] Đã lưu: D:\file\project\Sign-Language-Classification-v1.0\data\a\8.csv
Đã thoát.
✔️ Hoàn tất thu thập 'a' (8 mẫu).
